## 10-fold Cross-validation with Ridge regression

In [ ]:
# import libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn
from sklearn.linear_model import Ridge

In [ ]:
# define constants

folds = 10
lamda = np.array([0.01, 0.1, 1, 10, 100])

In [ ]:
# load training data

# load data from csv file
data_frame = pd.read_csv ('train.csv')
x_values = data_frame[['x1','x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8','x9','x10','x11','x12','x13']]
y_values = data_frame[['y']]
# visualize data
print(data_frame[:2])

In [ ]:
# data pre-processing (eg. standardize)

In [ ]:
# train model: 10-fold cross validation with ridge regression

In [ ]:
# load test data

In [ ]:
# prediction for test data

In [ ]:
# save predictions